In [1]:
import pandas as pd
import numpy as np
import sys

daily_df=pd.read_excel('input_data_2.xlsx', sheet_name = "Sheet4")
curve_df = pd.read_excel('VEA.xlsx')

In [2]:
daily_df

,Month,Discharge(m3/s),K_Value,Ke,Energy
0,2010-01-01,1.379582,1.72,0.05332,11261.5
1,2010-02-01,1.383972,2.18,0.06104,9901.0
2,2010-03-01,1.454920,4.42,0.13702,10315.0
3,2010-04-01,1.406824,7.43,0.22290,8738.5
4,2010-05-01,1.501326,8.20,0.25420,6175.0
...,...,...,...,...,...
151,2022-08-01,7.336924,1.90,0.05890,5769.0
152,2022-09-01,2.913782,1.87,0.05610,2617.0
153,2022-10-01,1.659041,2.33,0.07223,273.0
154,2022-11-01,1.518059,2.07,0.06210,2907.5


In [3]:
def convert_cms_to_mcm(val):
    result = val*30.*24.*60.*60./1.e6
    return result

In [4]:
def predict_y_from_x(dataframe, x_column, y_column, degree, x_to_predict):
    x_values = dataframe[x_column].values
    y_values = dataframe[y_column].values
    
    coefficients = np.polyfit(x_values, y_values, degree)
    poly_equation = np.poly1d(coefficients)
    
    y_predicted = poly_equation(x_to_predict)
    
    
    return y_predicted

x_column = 'Volume'
y_column = 'Elevation'
degree = 10


In [5]:
def predict_a_from_h(dataframe, x1_column, y1_column, degree1, x1_to_predict):
    x1_values = dataframe[x1_column].values
    y1_values = dataframe[y1_column].values
    
    coefficients = np.polyfit(x1_values, y1_values, degree1)
    poly_equation = np.poly1d(coefficients)
    
    a_predicted = poly_equation(x1_to_predict)
    
    
    return a_predicted

x1_column = 'Elevation'
y1_column = 'Area'
degree1=6

In [6]:
def predict_a_from_v(dataframe, x2_column, y2_column, degree2, x2_to_predict):
    x2_values = dataframe[x2_column].values
    y2_values = dataframe[y2_column].values
    
    coefficients = np.polyfit(x2_values, y2_values, degree2)
    poly_equation = np.poly1d(coefficients)
    
    a_predicted = poly_equation(x2_to_predict)
    
    
    return a_predicted

x2_column = 'Volume'
y2_column = 'Area'
degree2=10

In [7]:
def predict_v_from_h(dataframe, x3_column, y3_column, degree3, x3_to_predict):
    x3_values=dataframe[x3_column].values
    y3_values= dataframe[y3_column].values
    
    coefficients = np.polyfit(x3_values, y3_values, degree3)
    poly_equation = np.poly1d(coefficients)
    
    v_predicted = poly_equation(x3_to_predict)
    
    return v_predicted

In [8]:
x3_column = 'Elevation'
y3_column = 'Volume'
degree3 = 6
x3_to_predict = float(input ("Enter initial water level:  "))


Enter initial water level:  1514.37


In [9]:
initial_storage = predict_v_from_h(curve_df, x3_column, y3_column, degree3, x3_to_predict)
initial_storage

34.476346492767334

In [10]:
def convert_eg_to_demand(energy , elevation):
    head = elevation - 916
    result = (energy*3600)/(0.85*9810*head)
    return result

In [11]:
daily_df['Inflow']= convert_cms_to_mcm(daily_df['Discharge(m3/s)'].to_numpy())


In [12]:
def reservoir_simulation(df, initial_storage, storage_capacity):
    inflow=df['Inflow'].to_numpy()
    Energy=df['Energy'].to_numpy()
    K=df['Ke'].to_numpy()
    N=len(inflow)
    
    demand=np.zeros(N)
    storage=np.zeros(N)
    height=np.zeros(N)
    area=np.zeros(N)
    trial_evaporation=np.zeros(N)
    evap=np.zeros(N)
    delivery=np.zeros(N)
    spill=np.zeros(N)
    outflow=np.zeros(N)
    energy_generated=np.zeros(N)

    
    storage[0]=initial_storage
    
    
    for i in range(N):
        
        x_to_predict =storage[i]
        height[i] = predict_y_from_x(curve_df, x_column, y_column, degree, x_to_predict)
        
        x2_to_predict = storage[i]
        area[i]= predict_a_from_v(curve_df, x2_column, y2_column, degree2, x2_to_predict)
        
        elevation = height[i]
        energy= Energy[i]
        demand[i]= convert_eg_to_demand (energy , elevation)
        
        trial_evaporation[i] = (K[i] * area[i])/1.e6
        
        available_water=inflow[i]+storage[i]-trial_evaporation[i]
        
        if available_water>demand[i]:
            delivery[i]=demand[i]
        else:
            delivery[i]=available_water
        
        delivery[i] = max(0, delivery[i])
        
        trial_storage=storage[i]+inflow[i]-delivery[i]-trial_evaporation[i]
        trial_storage = max(0, min(trial_storage, storage_capacity))
        
        if trial_storage>storage_capacity:
            spill[i]= trial_storage-storage_capacity
            if i< N-1:
                storage[i+1]=storage_capacity
        else:
            spill[i]=0
            if i<N-1:
                storage[i+1]=trial_storage
        
       
        
        
        area[i]=predict_a_from_v(curve_df, x2_column, y2_column, degree2, storage[i])
        if i<N-1:
            x2_to_predict=storage[i+1]
            area[i+1]=predict_a_from_v(curve_df, x2_column, y2_column, degree2, x2_to_predict)
        
        if i<N-1:
            while True:
                a=area[i]
         
                b=area[i+1]
                c=(a+b)*0.5
                evap[i] = K[i]*c/1.e6
                available_water=inflow[i]+storage[i]- evap[i]
        
                if available_water>demand[i]:
                    delivery[i]=demand[i]
                else:
                    delivery[i]=available_water
                    
                delivery[i] = max(0, delivery[i])

        
                trial_storage=storage[i]+inflow[i]-delivery[i]-evap[i]
                trial_storage = max(0, trial_storage)
        
                if trial_storage>storage_capacity:
                    spill[i]= trial_storage-storage_capacity
                    storage[i+1]=storage_capacity
                else:
                    spill[i]=0
                    storage[i+1]=trial_storage
        
                area[i+1]=predict_a_from_v(curve_df,x2_column, y2_column, degree2, storage[i+1])
         
            
                if abs(b-area[i+1])<0.0001:
                    break
                
            
        outflow[i]=spill[i]+delivery[i]
        energy_generated[i]=(0.85*9810*delivery[i]*(height[i]-916))/3600

        
            
        
      
    df['demand']=demand
    df['storage']=storage
    df['Height']=height
    df['Area']=area
    df['Evap Loss']=trial_evaporation
    df['Evap']=evap
    df['Delivery']=delivery
    df['Spill']=spill
    df['Outflow']=outflow
    df['Generated'] = energy_generated
    
    return df

In [13]:
maximum_capacity=67.5
daily_df=reservoir_simulation(daily_df, initial_storage, maximum_capacity)

In [14]:
daily_df

,Month,Discharge(m3/s),K_Value,Ke,Energy,Inflow,demand,storage,Height,Area,Evap Loss,Evap,Delivery,Spill,Outflow,Generated
0,2010-01-01,1.379582,1.72,0.05332,11261.5,3.575877,8.124314,34.476346,1514.444846,1.367725e+06,0.072927,0.069274,8.124314,0.0,8.124314,11261.5
1,2010-02-01,1.383972,2.18,0.06104,9901.0,3.587255,7.187420,29.858635,1510.731001,1.230702e+06,0.075122,0.071438,7.187420,0.0,7.187420,9901.0
2,2010-03-01,1.454920,4.42,0.13702,10315.0,3.771154,7.528759,26.187032,1507.507700,1.110000e+06,0.152092,0.143314,7.528759,0.0,7.528759,10315.0
3,2010-04-01,1.406824,7.43,0.22290,8738.5,3.646487,6.417679,22.286112,1503.859388,9.818707e+05,0.218859,0.208312,6.417679,0.0,6.417679,8738.5
4,2010-05-01,1.501326,8.20,0.25420,6175.0,3.891437,4.558762,19.306608,1500.796235,8.872335e+05,0.225535,0.221980,4.558762,0.0,4.558762,6175.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,2022-08-01,7.336924,1.90,0.05890,5769.0,19.017308,4.469384,3.045963,1473.272334,3.558181e+05,0.020958,0.034988,4.469384,0.0,4.469384,5769.0
152,2022-09-01,2.913782,1.87,0.05610,2617.0,7.552523,1.938628,17.558900,1498.805856,8.322172e+05,0.046687,0.051643,1.938628,0.0,1.938628,2617.0
153,2022-10-01,1.659041,2.33,0.07223,273.0,4.300235,0.200220,23.121153,1504.665734,1.008897e+06,0.072873,0.077673,0.200220,0.0,0.200220,273.0
154,2022-11-01,1.518059,2.07,0.06210,2907.5,3.934809,2.119067,27.143495,1508.365419,1.141805e+06,0.070906,0.072691,2.119067,0.0,2.119067,2907.5


In [15]:
excel_file_path ='Sample_output2.xlsx'
daily_df.to_excel(excel_file_path, index=False)